In [1]:
import d2lzh as d2l
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time

(corpus_indices, char_to_idx, idx_to_char,
 vocab_size) = d2l.load_data_jay_lyrics()

In [2]:
# 构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层rnn_layer
num_hiddens = 256
rnn_layer = rnn.RNN(num_hiddens)
rnn_layer.initialize()

In [3]:
batch_size = 2
# 返回初始化的隐藏状态列表
state = rnn_layer.begin_state(batch_size=batch_size)
# 形状为(隐藏层个数, 批量大小, 隐藏单元个数)
state[0].shape

(1, 2, 256)

In [8]:
# 前向计算的“输出”，形状为(时间步数, 批量大小, 隐藏单元个数)。
num_steps = 35
X = nd.random.uniform(shape=(num_steps, batch_size, vocab_size))
Y, state_new = rnn_layer(X, state)
X.shape, Y.shape, len(state_new), state_new[0].shape

((35, 2, 1027), (35, 2, 256), 1, (1, 2, 256))

In [9]:
# 首先将输入数据使用one-hot向量表示后输入到rnn_layer中，然后使用全连接输出层得到输出。
class RNNModel(nn.Block):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.dense = nn.Dense(vocab_size)

    def forward(self, inputs, state):
        # 将输入转置成(num_steps, batch_size)后获取one-hot向量表示
        X = nd.one_hot(inputs.T, self.vocab_size)
        Y, state = self.rnn(X, state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

In [10]:
# 预测函数
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char,
                      char_to_idx):
    # 使用model的成员函数来初始化隐藏状态
    state = model.begin_state(batch_size=1, ctx=ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1, 1))
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [12]:
ctx = d2l.try_gpu()
model = RNNModel(rnn_layer, vocab_size)
model.initialize(force_reinit=True, ctx=ctx)
predict_rnn_gluon('分开', 20, model, vocab_size, ctx, idx_to_char, char_to_idx)

'分开吴辈颁想爸自蝙药驳蝙延壁椅誓蝙寞蝙办寞背'

In [13]:
# 本函数已保存在d2lzh包中方便以后使用
def train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx, force_reinit=True, init=init.Normal(0.01))
    trainer = gluon.Trainer(model.collect_params(), 'sgd',
                            {'learning_rate': lr, 'momentum': 0, 'wd': 0})

    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        # 相邻采样
        data_iter = d2l.data_iter_consecutive(
            corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        for X, Y in data_iter:
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(output, y).mean()
            l.backward()
            # 梯度裁剪
            params = [p.data() for p in model.collect_params().values()]
            d2l.grad_clipping(params, clipping_theta, ctx)
            trainer.step(1)  # 因为已经误差取过均值，梯度不用再做平均
            l_sum += l.asscalar() * y.size
            n += y.size

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_gluon(
                    prefix, pred_len, model, vocab_size, ctx, idx_to_char,
                    char_to_idx))

In [14]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 79.373454, time 0.05 sec
 - 分开 我不能这生写 一哼的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏柔的
 - 不分开 我想我这样写 一哼的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏柔的
epoch 100, perplexity 13.340477, time 0.05 sec
 - 分开 娘子我怕抽太是不场道剧 我想我这辈子注定知前人不多 娘子我抬起你是一场悲剧 我想我这辈子注定知个人
 - 不分开有一你的手不放中走过风想说么 我想我将辈子注定知个人不著 娘我的外步在枯的牧了在人 一坏的让我疯狂的
epoch 150, perplexity 3.998861, time 0.05 sec
 - 分开 我以 我些 我不能再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开有的美 吸血准备来袭 我怀念声国了 这静依的太快 像会是著我 你口盯口不投 又身会掩 我该好好生我 
epoch 200, perplexity 2.288218, time 0.05 sec
 - 分开 什么兵器最喜日 双截棍柔武休刚 学少林跟武嵩山 学少林跟武当 快使用双截棍 哼哼哈兮 快使用双截棍
 - 不分开 不是再这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 难说你手不会痛吗 不要再这样打我妈妈 难
epoch 250, perplexity 1.942336, time 0.05 sec
 - 分开 我一多再想牵着你的手不放开 一可不能以简远单单没有伤害 你 靠着我的肩膀 你 在我胸口睡著 像这样
 - 不分开觉不着我 想要起么睡勉 篮黄忙碌地排了 我都能伊斯坦堡 就像是童话故事 有教凶多 再有一碗热粥 配上
